In [111]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import pretraitement

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [112]:
DATA_x = pd.read_csv('data/X_train_G3tdtEn.csv', delimiter = ',',encoding = 'latin1')
DATA_y = pd.read_csv('data/Y_train_2_XPXJDyy.csv', delimiter = ',',encoding = 'latin1')

C:\Users\kille\AppData\Local\Temp\ipykernel_15388\2876525926.py:1: DtypeWarning: Columns (21,22,23,24,69,70,71,72,93,94,95,96,97,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  DATA_x = pd.read_csv('data/X_train_G3tdtEn.csv', delimiter = ',',encoding = 'latin1')


In [113]:
cash_price_columns = ["cash_price" + str(i) for i in range(1, 4)]
nbr_columns = ["Nbr_of_prod_purchas" + str(i) for i in range(1, 4)]

selection_colonnes = ['ID'] + cash_price_columns
df_prices = DATA_x[selection_colonnes]
df_prices = pd.merge(df_prices, DATA_y, on='ID')
df_prices = df_prices.drop("index",axis=1)
df_prices

,ID,cash_price1,cash_price2,cash_price3,fraud_flag
0,85517,889.0,NaN,NaN,0
1,51113,409.0,NaN,NaN,0
2,83008,1399.0,NaN,NaN,0
3,78712,689.0,119.0,NaN,0
4,77846,1199.0,NaN,NaN,0
...,...,...,...,...,...
92785,21243,299.0,7.0,NaN,0
92786,45891,898.0,NaN,NaN,0
92787,42613,519.0,259.0,949.0,0
92788,43567,1099.0,2099.0,NaN,0


In [114]:
print("Nombre de perfect : ", df_prices[df_prices['fraud_flag'] == 0].shape)
print("Nombre de fraudes : ", df_prices[df_prices['fraud_flag'] == 1].shape)

Nombre de perfect :  (91471, 5)
Nombre de fraudes :  (1319, 5)


# Test : sous-échantillonnage des achats non frauduleux

Actuellement, on a un rapport **99%** vs **1%** d'achats frauduleux 

Sans forcément passer à 50% / 50%, on peut essayer de faire du sous échantillonnage avec plusieurs taux, comme 90/10 ou 80/20

In [115]:
df_prices.columns

Index(['ID', 'cash_price1', 'cash_price2', 'cash_price3', 'fraud_flag'], dtype='object')

In [116]:
Y = df_prices[['ID','fraud_flag']]
X = df_prices.drop("fraud_flag", axis=1)
X = X.fillna(0)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

In [117]:

nb_perfect_X = df_prices[df_prices['fraud_flag'] == 0].shape[0]
nb_fraude_X = df_prices[df_prices['fraud_flag'] == 1].shape[0]
nb_perfect_train = Y_train['fraud_flag'].value_counts(normalize=False)[0]
nb_fraude_train = Y_train['fraud_flag'].value_counts(normalize=False)[1]
nb_perfect_test = Y_test['fraud_flag'].value_counts(normalize=False)[0]
nb_fraude_test = Y_test['fraud_flag'].value_counts(normalize=False)[1]

print("Nombre de perfect : ", nb_perfect_X, " (", "{:.2f}".format(100 - 100*nb_fraude_X/nb_perfect_X), "%)")
print("Nombre de perfect : ", nb_fraude_X, " (", "{:.2f}".format(100*nb_fraude_X/nb_perfect_X), "%)")
print("Nombre de perfect dans Y_train : ", nb_perfect_train, " (", "{:.2f}".format(100 - 100*nb_fraude_train/nb_perfect_train), "%)")
print("Nombre de fraudes dans Y_train : ", nb_fraude_train, " (", "{:.2f}".format(100*nb_fraude_train/nb_perfect_train), "%)")
print("Nombre de perfect dans Y_test : ", nb_perfect_test, " (", "{:.2f}".format(100 - 100*nb_fraude_test/nb_perfect_test), "%)")
print("Nombre de fraudes dans Y_test : ", nb_fraude_test, " (", "{:.2f}".format(100*nb_fraude_test/nb_perfect_test), "%)")


Nombre de perfect :  91471  ( 98.56 %)
Nombre de perfect :  1319  ( 1.44 %)
Nombre de perfect dans Y_train :  73185  ( 98.57 %)
Nombre de fraudes dans Y_train :  1047  ( 1.43 %)
Nombre de perfect dans Y_test :  18286  ( 98.51 %)
Nombre de fraudes dans Y_test :  272  ( 1.49 %)


On va faire sous-échantillonnage uniquement sur la base de train, pour tester la robustesse sur la base de test

In [118]:
from sklearn.utils import resample

XY_train = pd.merge(X_train, Y_train, on='ID')

# Séparation des données frauduleuses et non-frauduleuses
fraud = XY_train[XY_train['fraud_flag'] == 1]
not_fraud = XY_train[XY_train['fraud_flag'] == 0]

# Sous-échantillonnage de la classe majoritaire (non-frauduleuse)
not_fraud_sampled = resample(not_fraud, replace=False, n_samples=10*len(fraud), random_state=42)

# Concaténation des données frauduleuses et non-frauduleuses sous-échantillonnées
data_sampled = pd.concat([fraud, not_fraud_sampled])

# Mélange des données pour éviter les biais
data_sampled = data_sampled.sample(frac=1, random_state=42)

In [119]:
nb_perfect_sampled = data_sampled['fraud_flag'].value_counts(normalize=False)[0]
nb_fraude_sampled = data_sampled['fraud_flag'].value_counts(normalize=False)[1]

print("Nombre de perfect dans data_sampled : ", nb_perfect_sampled, " (", "{:.2f}".format(100 - 100*nb_fraude_sampled/nb_perfect_sampled), "%)")
print("Nombre de fraudes dans data_sampled : ", nb_fraude_sampled, " (", "{:.2f}".format(100*nb_fraude_sampled/nb_perfect_sampled), "%)")

Nombre de perfect dans data_sampled :  1047  ( 0.00 %)
Nombre de fraudes dans data_sampled :  1047  ( 100.00 %)


In [120]:
Y_train = data_sampled['fraud_flag']
X_train = data_sampled.drop('ID',axis=1)
X_train = X_train.drop("fraud_flag", axis=1)

In [121]:
Y_test

,ID,fraud_flag
46180,107424,0
25031,74645,0
32798,102653,0
38656,23831,0
16271,60691,0
...,...,...
14569,96137,0
70048,84464,0
39431,62549,0
34291,89894,0


In [122]:
# Initialisation d'un modèle de forêt aléatoire
clf = RandomForestClassifier(n_estimators=250, random_state=42)

# Apprentissage du modèle sur l'ensemble de formation
clf.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=250, random_state=42)

In [126]:
X_test = X_test.drop("ID", axis=1)

In [127]:

# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

In [128]:
from sklearn.metrics import average_precision_score

average_precision = average_precision_score(Y_test, y_pred)

print(average_precision)

0.02727411285244581
